In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
import plotly.graph_objects as graph_objects
import findspark

findspark.init()

# spark_context = SparkContext.getOrCreate()
spark_context = SparkContext('local', 'R2_A3')
spark_session = SparkSession(spark_context)
sqlContext = SQLContext(spark_context)

/usr/local/spark/python/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [31]:
text_file = sqlContext.read.format('com.databricks.spark.csv').\
    options(header='true', inferschema='true', quote='"', delimiter=',').\
    load('./work/data/Covid19.csv')

rddfiltro = text_file.rdd.map(tuple)
rddGen = rddfiltro.map(lambda word: (word[3], word[5]))
rddGen.take(10)

[('September', 0),
 ('September', 125),
 ('September', 47),
 ('September', 0),
 ('September', 17),
 ('September', 40),
 ('September', 99),
 ('September', 75),
 ('September', 35),
 ('September', 34)]

In [32]:
rddConteo = rddGen.reduceByKey(lambda a, b: a+b)
print(f'Conteo total -> {rddConteo.collect()}')

Conteo total -> [('September', 2374517), ('August', 3584040), ('July', 3288723), ('June', 2527885), ('May', 1555025), ('March', 324799), ('April', 868927), ('February', 73635), ('January', 9758)]


In [33]:
rddOrden = spark_context.parallelize(
    rddConteo.sortBy(lambda a: a[1], True).take(5))
print(f'meses con menos casos de COVID -> {rddOrden.collect()}')

meses con menos casos de COVID -> [('January', 9758), ('February', 73635), ('March', 324799), ('April', 868927), ('May', 1555025)]


In [34]:
rddNombres = rddOrden.map(lambda x: (x[0]))
print(rddNombres.collect())

rddTotales = rddOrden.map(lambda x: (x[1]))
print(rddTotales.collect())

['January', 'February', 'March', 'April', 'May']
[9758, 73635, 324799, 868927, 1555025]


In [35]:
graph = graph_objects.Figure(
    data=graph_objects.Pie(
        labels=rddNombres.collect(),
        values=rddTotales.collect()
    ))

graph.update_layout(
    title_text='Meses con menos casos de COVID',
    title_font_size=30)

graph.update_traces(
    hoverinfo='label+percent',
    textinfo='value',
    textfont_size=20)

graph.write_html('./work/reports/R2_A3.html', auto_open=True)